# CNN 기반 모델

In [ ]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

## Simple EfficientNet-B0

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

In [ ]:
# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

In [ ]:
# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

In [ ]:
# 하이퍼파라미터 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_path = '../data/'
model_name = 'efficientnet_b0'
img_size = 384
LR = 1e-3
EPOCHS = 30
BATCH_SIZE = 32
num_workers = 4

# 데이터 증강 설정
train_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [ ]:
# 데이터 로드 및 분할
df = pd.read_csv("../data/train.csv")
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

train_dataset = ImageDataset(train_df, "../data/train_preprocessed/", transform=train_transform)
val_dataset = ImageDataset(val_df, "../data/train_preprocessed/", transform=val_transform)
test_dataset = ImageDataset("../data/sample_submission.csv", "../data/test_preprocessed/", transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

In [ ]:
# 모델 설정
model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [ ]:
from torchsummary import summary

# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)
    
# 모델 구조 출력
print(f"\nModel structure of {model_name}:")
print_model_summary(model, (3, img_size, img_size))

# 모델 아키텍처 출력
print("\nModel architecture:")
print(model)

In [ ]:
# 학습 루프
best_val_f1 = 0
for epoch in range(EPOCHS):
    train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
    val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
    scheduler.step()

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_model.pth")

In [ ]:
# 테스트 데이터 추론
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
preds_list = []

for image, _ in tqdm(test_loader):
    image = image.to(device)
    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

In [ ]:
# 결과 저장
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv("pred.csv", index=False)
print("Prediction completed and saved to pred.csv")

## EfficientNet-B4

In [ ]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
import gc

gc.collect()
torch.cuda.empty_cache()


# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((380, 380, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)

# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    model_name = 'efficientnet_b4'
    img_size = 380  # EfficientNet-B4에 적합한 이미지 크기
    LR = 5e-4  # 학습률 조정
    EPOCHS = 30
    BATCH_SIZE = 16  # 배치 크기 감소
    num_workers = 4

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # 데이터 로드 및 분할
    df = pd.read_csv(os.path.join(data_path, "train_correct_labeling.csv"))
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform)
    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    # 모델 구조 출력
    print(f"\nModel structure of {model_name}:")
    print_model_summary(model, (3, img_size, img_size))

    # 학습 루프
    best_val_f1 = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
        scheduler.step()

        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")

    # 테스트 데이터 추론
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    preds_list = []

    for image, _ in tqdm(test_loader):
        image = image.to(device)
        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

    # 결과 저장
    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    pred_df.to_csv("pred.csv", index=False)
    print("Prediction completed and saved to pred.csv")

## ConvNext V2 Large 모델

In [ ]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
import gc

gc.collect()
torch.cuda.empty_cache()

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)

# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    model_name = 'convnextv2_large'
    img_size = 224  # ConvNeXt V2 Large에 적합한 이미지 크기
    LR = 1e-4  # 학습률 조정
    EPOCHS = 30
    BATCH_SIZE = 32  # 배치 크기 조정
    num_workers = 4

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # 데이터 로드 및 분할
    df = pd.read_csv(os.path.join(data_path, "train_correct_labeling.csv"))
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform)
    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    # 모델 구조 출력
    print(f"\nModel structure of {model_name}:")
    print_model_summary(model, (3, img_size, img_size))

    # 학습 루프
    best_val_f1 = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
        scheduler.step()

        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "convNext_model.pth")

    # 테스트 데이터 추론
    model.load_state_dict(torch.load("convNext_model.pth"))
    model.eval()
    preds_list = []

    for image, _ in tqdm(test_loader):
        image = image.to(device)
        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

    # 결과 저장
    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    pred_df.to_csv("pred.csv", index=False)
    print("Prediction completed and saved to pred.csv")

## convNext v2 + fine tunning

In [3]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
import gc

gc.collect()
torch.cuda.empty_cache()

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []
    misclassified = []

    with torch.no_grad():
        for i, (image, targets) in enumerate(loader):
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_np = preds.argmax(dim=1).detach().cpu().numpy()
            targets_np = targets.detach().cpu().numpy()
            preds_list.extend(preds_np)
            targets_list.extend(targets_np)

            # 오분류된 데이터의 인덱스 저장
            misclassified.extend(np.where(preds_np != targets_np)[0] + i * loader.batch_size)

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1, misclassified

# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)

# EarlyStopping 클래스 정의
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    model_name = 'convnextv2_large'
    img_size = 224
    LR = 1e-4
    EPOCHS = 30
    BATCH_SIZE = 32
    num_workers = 4

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # 데이터 로드 및 분할
    df = pd.read_csv(os.path.join(data_path, "train_correct_labeling.csv"))
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train/"), transform=train_transform)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train/"), transform=val_transform)
    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test/"), transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    # 모델 구조 출력
    print(f"\nModel structure of {model_name}:")
    print_model_summary(model, (3, img_size, img_size))

    # EarlyStopping 초기화
    early_stopping = EarlyStopping(patience=5, min_delta=0.001)

    # 학습 루프
    best_val_f1 = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc, val_f1, misclassified = validate(val_loader, model, loss_fn, device)
        scheduler.step()

        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "convNext_model.pth")

        # Early Stopping 체크
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

    # 오분류된 데이터로 fine-tuning
    misclassified_df = val_df.iloc[misclassified]
    misclassified_dataset = ImageDataset(misclassified_df, os.path.join(data_path, "train/"), transform=train_transform)
    misclassified_loader = DataLoader(misclassified_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)

    print("\nFine-tuning with misclassified data")
    for epoch in range(10):  # 10 에폭 동안 fine-tuning
        train_loss, train_acc, train_f1 = train_one_epoch(misclassified_loader, model, optimizer, loss_fn, device)
        print(f"Fine-tuning Epoch {epoch+1}/5")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")

    # 최종 모델 저장
    torch.save(model.state_dict(), "convNext_model_final.pth")

    # 테스트 데이터 추론
    model.load_state_dict(torch.load("convNext_model_final.pth"))
    model.eval()
    preds_list = []

    for image, _ in tqdm(test_loader):
        image = image.to(device)
        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

    # 결과 저장
    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    pred_df.to_csv("conv_fine_pred.csv", index=False)
    print("Prediction completed and saved to pred.csv")


Model structure of convnextv2_large:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 192, 56, 56]           9,408
       LayerNorm2d-2          [-1, 192, 56, 56]             384
          Identity-3          [-1, 192, 56, 56]               0
            Conv2d-4          [-1, 192, 56, 56]           9,600
         LayerNorm-5          [-1, 56, 56, 192]             384
            Linear-6          [-1, 56, 56, 768]         148,224
              GELU-7          [-1, 56, 56, 768]               0
           Dropout-8          [-1, 56, 56, 768]               0
GlobalResponseNorm-9          [-1, 56, 56, 768]           1,536
           Linear-10          [-1, 56, 56, 192]         147,648
          Dropout-11          [-1, 56, 56, 192]               0
GlobalResponseNormMlp-12          [-1, 56, 56, 192]               0
         Identity-13          [-1, 192, 56, 56]              

Loss: 0.2666: 100%|██████████| 40/40 [00:29<00:00,  1.36it/s]


Epoch 1/30
Train Loss: 0.9142, Train Acc: 0.7142, Train F1: 0.6951
Val Loss: 0.3429, Val Acc: 0.8758, Val F1: 0.8352


Loss: 0.0162: 100%|██████████| 40/40 [00:29<00:00,  1.35it/s]


Epoch 2/30
Train Loss: 0.2789, Train Acc: 0.8941, Train F1: 0.8816
Val Loss: 0.3312, Val Acc: 0.8662, Val F1: 0.8275


...

Fine-tuning Epoch 9/5
Train Loss: 0.1177, Train Acc: 1.0000, Train F1: 1.0000


Loss: 0.1283: 100%|██████████| 1/1 [00:00<00:00,  1.38it/s]


Fine-tuning Epoch 10/5
Train Loss: 0.1283, Train Acc: 0.9474, Train F1: 0.9481


/tmp/ipykernel_160351/3141216663.py:227: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("convNext_model_final.pth"))
100%|██████████| 99/99 [

Prediction completed and saved to pred.csv


## convNext v2 + new augmentaion + K-Fold finetunning + ensemble

In [2]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
import gc

gc.collect()
torch.cuda.empty_cache()

class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []
    misclassified = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_np = preds.argmax(dim=1).detach().cpu().numpy()
            targets_np = targets.detach().cpu().numpy()
            preds_list.extend(preds_np)
            targets_list.extend(targets_np)

            misclassified.extend(np.where(preds_np != targets_np)[0])

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1, misclassified

def print_model_summary(model, input_size):
    summary(model, input_size)

class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    model_name = 'convnextv2_large'
    img_size = 224
    LR = 1e-4
    EPOCHS = 30
    BATCH_SIZE = 32
    num_workers = 4
    n_splits = 5

    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.OneOf([
            A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
            A.RandomGamma(gamma_limit=(80, 120), p=0.5),
        ], p=0.5),
        A.OneOf([
            A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
            A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.5),
        ], p=0.5),
        A.OneOf([
            A.MotionBlur(blur_limit=3, p=0.5),
            A.MedianBlur(blur_limit=3, p=0.5),
            A.GaussianBlur(blur_limit=3, p=0.5),
        ], p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=5, border_mode=0, p=0.5),
        A.CoarseDropout(max_holes=8, max_height=img_size//20, max_width=img_size//20, min_holes=5, fill_value=255, p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    df = pd.read_csv(os.path.join(data_path, "train_correct_labeling.csv"))
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['target']), 1):
        print(f"\nFold {fold}")

        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)

        train_dataset = ImageDataset(train_df, os.path.join(data_path, "train/"), transform=train_transform)
        val_dataset = ImageDataset(val_df, os.path.join(data_path, "train/"), transform=val_transform)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

        model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
        scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

        if fold == 1:
            print(f"\nModel structure of {model_name}:")
            print_model_summary(model, (3, img_size, img_size))

        best_val_f1 = 0
        misclassified_data = []
        early_stopping = EarlyStopping(patience=5, min_delta=0.001)

        for epoch in range(EPOCHS):
            train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
            val_loss, val_acc, val_f1, misclassified = validate(val_loader, model, loss_fn, device)
            scheduler.step()

            print(f"Epoch {epoch+1}/{EPOCHS}")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                torch.save(model.state_dict(), f"convNext_model_fold{fold}.pth")

            misclassified_data.extend(val_df.iloc[misclassified].index)

            early_stopping(val_loss)
            if early_stopping.early_stop:
                print("Early stopping")
                break

        misclassified_df = df.loc[misclassified_data]
        misclassified_dataset = ImageDataset(misclassified_df, os.path.join(data_path, "train/"), transform=train_transform)
        misclassified_loader = DataLoader(misclassified_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)

        print("\nFine-tuning with misclassified data")
        for epoch in range(5):
            train_loss, train_acc, train_f1 = train_one_epoch(misclassified_loader, model, optimizer, loss_fn, device)
            print(f"Epoch {epoch+1}/5")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")

        torch.save(model.state_dict(), f"convNext_model_fold{fold}_final.pth")

    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test/"), transform=val_transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    ensemble_preds = []
    for fold in range(1, n_splits + 1):
        model.load_state_dict(torch.load(f"convNext_model_fold{fold}_final.pth"))
        model.eval()
        fold_preds = []

        for image, _ in tqdm(test_loader, desc=f"Predicting Fold {fold}"):
            image = image.to(device)
            with torch.no_grad():
                preds = model(image)
            fold_preds.extend(preds.softmax(dim=1).detach().cpu().numpy())

        ensemble_preds.append(fold_preds)

    final_preds = np.mean(ensemble_preds, axis=0).argmax(axis=1)

    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = final_preds
    pred_df.to_csv("pred_ensemble.csv", index=False)
    print("Ensemble prediction completed and saved to pred_ensemble.csv")

/opt/conda/lib/python3.10/site-packages/albumentations/augmentations/blur/transforms.py:184: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(



Fold 1

Model structure of convnextv2_large:
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 192, 56, 56]           9,408
       LayerNorm2d-2          [-1, 192, 56, 56]             384
          Identity-3          [-1, 192, 56, 56]               0
            Conv2d-4          [-1, 192, 56, 56]           9,600
         LayerNorm-5          [-1, 56, 56, 192]             384
            Linear-6          [-1, 56, 56, 768]         148,224
              GELU-7          [-1, 56, 56, 768]               0
           Dropout-8          [-1, 56, 56, 768]               0
GlobalResponseNorm-9          [-1, 56, 56, 768]           1,536
           Linear-10          [-1, 56, 56, 192]         147,648
          Dropout-11          [-1, 56, 56, 192]               0
GlobalResponseNormMlp-12          [-1, 56, 56, 192]               0
         Identity-13          [-1, 192, 56, 56]      

Loss: 0.3000: 100%|██████████| 40/40 [00:29<00:00,  1.36it/s]


Epoch 1/30
Train Loss: 0.8191, Train Acc: 0.7524, Train F1: 0.7394
Val Loss: 0.3038, Val Acc: 0.8790, Val F1: 0.8583


Loss: 0.0397: 100%|██████████| 40/40 [00:29<00:00,  1.35it/s]


...

Epoch 4/5
Train Loss: 0.0005, Train Acc: 1.0000, Train F1: 1.0000


Loss: 0.0003: 100%|██████████| 9/9 [00:06<00:00,  1.37it/s]


Epoch 5/5
Train Loss: 0.0006, Train Acc: 1.0000, Train F1: 1.0000


/tmp/ipykernel_160351/481695052.py:234: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"convNext_model_fold{fold}_final.pth"))
Predicting Fo

Ensemble prediction completed and saved to pred_ensemble.csv


## Hyper Parameters Tunning With CNN Based Model

### ConvNeXt V2 Large 모델 + Optuna

In [ ]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
import optuna
import gc

gc.collect()
torch.cuda.empty_cache()

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)

# Optuna를 이용한 하이퍼파라미터 최적화 함수
def objective(trial):
    # 하이퍼파라미터 탐색 공간 정의
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)

    # 데이터 로더 생성
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델, 손실 함수, 옵티마이저 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    # 학습 및 검증
    best_val_f1 = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
        scheduler.step()

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1

    return best_val_f1

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_path = '../data/'
model_name = 'convnextv2_large'
img_size = 224  # ConvNeXt V2에 적합한 이미지 크기
EPOCHS = 30
num_workers = 4

# 데이터 증강 설정
train_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터 로드 및 분할
df = pd.read_csv(os.path.join(data_path, "train.csv"))
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform)
val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform)
test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)

# Optuna를 이용한 하이퍼파라미터 최적화
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

best_params = study.best_params
print("Best hyperparameters:", best_params)

# 최적의 하이퍼파라미터로 최종 모델 학습
best_lr = best_params['lr']
best_batch_size = best_params['batch_size']
best_weight_decay = best_params['weight_decay']

train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=best_lr, weight_decay=best_weight_decay)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

# 모델 구조 출력
print(f"\nModel structure of {model_name}:")
print_model_summary(model, (3, img_size, img_size))

# 학습 루프
best_val_f1 = 0
for epoch in range(EPOCHS):
    train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
    val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
    scheduler.step()

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_model.pth")

# 테스트 데이터 추론
model.load_state_dict(torch.load("best_model.pth"))
model.eval()
preds_list = []

for image, _ in tqdm(test_loader):
    image = image.to(device)
    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# 결과 저장
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv("pred.csv", index=False)
print("Prediction completed and saved to pred.csv")

### ConvNext V2 Large + WanDB Sweep
- pip install wandb
- wandb login

In [ ]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import wandb
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary

# 데이터셋 클래스 정의
#class ImageDataset(Dataset):


# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
#def validate(loader, model, loss_fn, device):

# 모델 구조 출력 함수
#def print_model_summary(model, input_size):


# wandb sweep을 위한 학습 함수
def train():
    # wandb 초기화
    run = wandb.init(entity="cho") #사용자에 따라 자신의 도메인 네임 설정!!!
    config = wandb.config

    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    model_name = config.model_name
    img_size = config.img_size
    LR = config.learning_rate
    EPOCHS = config.epochs
    BATCH_SIZE = config.batch_size
    num_workers = 4

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # 데이터 로드 및 분할
    df = pd.read_csv(os.path.join(data_path, "train.csv"))
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=config.weight_decay)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    # 모델 구조 출력
    print(f"\nModel structure of {model_name}:")
    print_model_summary(model, (3, img_size, img_size))

    # 학습 루프
    best_val_f1 = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
        scheduler.step()

        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        # wandb에 로그 기록
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "train_f1": train_f1,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "val_f1": val_f1,
            "learning_rate": optimizer.param_groups[0]['lr']
        })

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            wandb.run.summary["best_val_f1"] = best_val_f1

    wandb.finish()

# wandb sweep 설정
sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'val_f1',
        'goal': 'maximize'
    },
    'parameters': {
        'model_name': {
            'values': ['convnextv2_large', 'efficientnet_b4']
        },
        'learning_rate': {
            'distribution': 'uniform',
            'min': 1e-5,
            'max': 1e-3
        },
        'batch_size': {
            'values': [16, 32, 64]
        },
        'img_size': {
            'values': [224, 256, 288]
        },
        'weight_decay': {
            'values': [1e-5, 1e-4, 1e-3]
        },
        'epochs': {
            'value': 30
        }
    }
}

# wandb sweep 실행 및 최고 성능 모델 찾기
sweep_id = wandb.sweep(sweep_config, project="cvmodel",entity="cho")
wandb.agent(sweep_id, train, count=30)

In [ ]:

# 최고 성능 모델의 설정 가져오기
api = wandb.Api()
sweep = api.sweep(f"dl-12/cvmodel/{sweep_id}")
best_run = sweep.best_run()
best_config = best_run.config

  
# 최고 성능 모델의 설정 사용
model_name = best_config['model_name']
img_size = best_config['img_size']
BATCH_SIZE = best_config['batch_size']
num_workers = 4

# 테스트 데이터 변환
test_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 테스트 데이터셋 및 데이터로더 생성
test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

# 최고 성능 모델 생성
model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# 테스트 데이터 추론
preds_list = []

for image, _ in tqdm(test_loader):
    image = image.to(device)
    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# 결과 저장
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv("pred.csv", index=False)
print("Prediction completed and saved to pred.csv")

# wandb에 결과 업로드
wandb.init(project="cvmodel", name="best_model_prediction", entity="cho")
wandb.config.update(best_config)
wandb.save("pred.csv")
wandb.finish()

# Transformer 기반 모델

## Swin Transformers

In [2]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [3]:
import os
import time
import timm

import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary


# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, min_f1_score=0.9):
        self.patience = patience
        self.min_delta = min_delta
        self.min_f1_score = min_f1_score
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, val_f1):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience and val_f1 >= self.min_f1_score:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            
# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)


# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    model_name = 'swin_large_patch4_window7_224'  # Swin Transformer Large 모델
    img_size = 224  # Swin Transformer에 적합한 이미지 크기
    LR = 2e-5  # 1e-4 에서 학습률 조정 : 기존이 더 좋음
    EPOCHS = 100 # 30에서 조정 : 기존이 더 좋음
    BATCH_SIZE = 32  # 배치 크기 조정
    num_workers = 4

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # 데이터 로드 및 분할
    df = pd.read_csv(os.path.join(data_path, "train_correct_labeling.csv"))
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform)
    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 모델 설정
    model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)
    # Early stopping 설정
    early_stopping = EarlyStopping(patience=5, min_delta=0.001, min_f1_score=0.93)

    # 모델 구조 출력
    # print(f"\nModel structure of {model_name}:")
    # print_model_summary(model, (3, img_size, img_size))

    # 학습 루프
    best_val_f1 = 0
    for epoch in range(EPOCHS):
        train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
        scheduler.step()

        print(f"Epoch {epoch+1}/{EPOCHS}")
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "swin_t_model.pth")
        
        # Early stopping 체크
        early_stopping(val_loss, val_f1)
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch+1} with F1 score: {val_f1:.4f}")
            break

    # 테스트 데이터 추론
    model.load_state_dict(torch.load("swin_t_model.pth"))
    model.eval()
    preds_list = []

    for image, _ in tqdm(test_loader):
        image = image.to(device)
        with torch.no_grad():
            preds = model(image)
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

    # 결과 저장
    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = preds_list
    pred_df.to_csv("pred_swin.csv", index=False)
    print("Prediction completed and saved to pred_swin.csv")


Loss: 1.2255: 100%|██████████| 40/40 [00:20<00:00,  1.92it/s]


Epoch 1/100
Train Loss: 2.1225, Train Acc: 0.4387, Train F1: 0.4000
Val Loss: 0.9821, Val Acc: 0.7452, Val F1: 0.6543


Loss: 0.2761: 100%|██████████| 40/40 [00:20<00:00,  1.95it/s]


Epoch 2/100
Train Loss: 0.7152, Train Acc: 0.8089, Train F1: 0.7800
Val Loss: 0.3570, Val Acc: 0.8758, Val F1: 0.8396


Loss: 0.2044: 100%|██████████| 40/40 [00:20<00:00,  1.94it/s]


...

Epoch 99/100
Train Loss: 0.0019, Train Acc: 0.9992, Train F1: 0.9993
Val Loss: 0.3706, Val Acc: 0.9363, Val F1: 0.9303


Loss: 0.0004: 100%|██████████| 40/40 [00:20<00:00,  1.92it/s]


Epoch 100/100
Train Loss: 0.0013, Train Acc: 1.0000, Train F1: 1.0000
Val Loss: 0.3706, Val Acc: 0.9363, Val F1: 0.9303


/tmp/ipykernel_17936/403596892.py:174: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("swin_t_model.pth"))
100%|██████████| 99/99 [00:18<00:0

Prediction completed and saved to pred_swin.csv


## Stratified k-fold cross validation 

In [30]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [31]:
import os
import time
import timm

import albumentations as A
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df.values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, min_f1_score=0.9):
        self.patience = patience
        self.min_delta = min_delta
        self.min_f1_score = min_f1_score
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, val_f1):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience and val_f1 >= self.min_f1_score:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            
# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)

# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    model_name = 'swin_large_patch4_window7_224'  # Swin Transformer Large 모델
    img_size = 224  # Swin Transformer에 적합한 이미지 크기
    LR = 2e-5  # 1e-4 에서 학습률 조정 : 기존이 더 좋음
    EPOCHS = 100 # 30에서 조정 : 기존이 더 좋음
    BATCH_SIZE = 32  # 배치 크기 조정
    num_workers = 4
    n_splits = 5  # Number of K-fold splits

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # 데이터 로드
    df = pd.read_csv(os.path.join(data_path, "train_correct_labeling.csv"))

    # Stratified K-Fold 설정
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # 전체 결과 저장
    final_preds_list = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['target'])):
        print(f"Fold {fold+1}/{n_splits}")
        
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]

        train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform)
        val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform)
        test_dataset = ImageDataset(pd.read_csv(os.path.join(data_path, "sample_submission.csv")), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

        # 모델 설정
        model = timm.create_model(model_name, pretrained=True, num_classes=17).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
        scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)
        # Early stopping 설정
        early_stopping = EarlyStopping(patience=5, min_delta=0.001, min_f1_score=0.93)

        # 모델 구조 출력
        # print(f"\nModel structure of {model_name}:")
        # print_model_summary(model, (3, img_size, img_size))

        # 학습 루프
        best_val_f1 = 0
        for epoch in range(EPOCHS):
            train_loss, train_acc, train_f1 = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
            val_loss, val_acc, val_f1 = validate(val_loader, model, loss_fn, device)
            scheduler.step()

            print(f"Epoch {epoch+1}/{EPOCHS}")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                torch.save(model.state_dict(), f"swin_t_model_fold_{fold}.pth")
            
            # Early stopping 체크
            early_stopping(val_loss, val_f1)
            if early_stopping.early_stop:
                print(f"Early stopping triggered at epoch {epoch+1} with F1 score: {val_f1:.4f}")
                break

        # 테스트 데이터 추론
        model.load_state_dict(torch.load(f"swin_t_model_fold_{fold}.pth"))
        model.eval()
        fold_preds_list = []

        for image, _ in tqdm(test_loader):
            image = image.to(device)
            with torch.no_grad():
                preds = model(image)
            fold_preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

        # 결과 저장
        final_preds_list.append(fold_preds_list)

    # 결과 앙상블
    final_preds = np.mean(np.array(final_preds_list), axis=0).astype(int)

    pred_df = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))
    pred_df['target'] = final_preds
    pred_df.to_csv("pred_swin_kfold.csv", index=False)
    print("Prediction completed and saved to pred_swin.csv")


Fold 1/5


Loss: 1.1379: 100%|██████████| 40/40 [00:20<00:00,  1.95it/s]


Epoch 1/100
Train Loss: 1.9977, Train Acc: 0.4865, Train F1: 0.4503
Val Loss: 0.8827, Val Acc: 0.7739, Val F1: 0.6936


Loss: 0.4577: 100%|██████████| 40/40 [00:20<00:00,  1.95it/s]


Epoch 2/100
Train Loss: 0.6732, Train Acc: 0.8049, Train F1: 0.7707
Val Loss: 0.4043, Val Acc: 0.8631, Val F1: 0.8179


...

Loss: 0.0135: 100%|██████████| 40/40 [00:20<00:00,  1.92it/s]


Epoch 10/100
Train Loss: 0.1205, Train Acc: 0.9546, Train F1: 0.9499
Val Loss: 0.1943, Val Acc: 0.9045, Val F1: 0.8985


Loss: 0.1353: 100%|██████████| 40/40 [00:20<00:00,  1.92it/s]


Epoch 11/100
Train Loss: 0.0913, Train Acc: 0.9745, Train F1: 0.9730
Val Loss: 0.2765, Val Acc: 0.9013, Val F1: 0.8873


Loss: 0.0312:  78%|███████▊  | 31/40 [00:16<00:04,  1.91it/s]

## Swin-T clustering & classification
- 이미지를 유사한 이미지로 5개로 그룹핑 하고 분석하는 모델
- early stoping 코드에서 강제적으로 f1 score boundary 를 줄수 있게 변경(분류하면 과소적합이 되는 구간이 있음.)

In [ ]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import gc

gc.collect()
torch.cuda.empty_cache()

# Swin-B 모델 로드
def load_swin_b_model(num_classes=None):
    model = timm.create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=num_classes)
    return model

# 특성 추출 함수
def extract_features(img_path, model):
    transform = A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])
    img = Image.open(img_path).convert('RGB')
    img = np.array(img)
    img = transform(image=img)['image']
    img = img.unsqueeze(0)
    
    with torch.no_grad():
        features = model.forward_features(img)
    return features.squeeze().numpy()

# 이미지 클러스터링 함수
def cluster_images(data_path, n_clusters=5):
    feature_extractor = load_swin_b_model(num_classes=None)
    feature_extractor.eval()
    
    image_files = [f for f in os.listdir(data_path) if f.endswith('.jpg') or f.endswith('.png')]
    features = []
    for img_file in tqdm(image_files, desc="Extracting features"):
        img_path = os.path.join(data_path, img_file)
        feature = extract_features(img_path, feature_extractor)
        features.append(feature.reshape(-1))  # Flatten the feature array
    
    features = np.array(features)
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(features_scaled)
    
    return dict(zip(image_files, clusters))

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None, cluster_dict=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform
        self.cluster_dict = cluster_dict

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        
        cluster = self.cluster_dict.get(name, -1) if self.cluster_dict else -1
        return img, target, cluster

# Early Stopping 클래스
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, min_f1_score=0.9):
        self.patience = patience
        self.min_delta = min_delta
        self.min_f1_score = min_f1_score
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, val_loss, val_f1):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.counter >= self.patience and val_f1 >= self.min_f1_score:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets, _ in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets, _ in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

# 모델 구조 출력 함수
def print_model_summary(model, input_size):
    summary(model, input_size)

# 메인 실행 코드
if __name__ == "__main__":
    # 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_path = '../data/'
    img_size = 224
    LR = 2e-5
    EPOCHS = 100
    BATCH_SIZE = 32
    num_workers = 4
    n_clusters = 3

    # 클러스터링 수행
    print("Clustering images...")
    cluster_dict = cluster_images(os.path.join(data_path, "train_preprocessed/"), n_clusters)

    # 데이터 증강 설정
    train_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    val_transform = A.Compose([
        A.Resize(height=img_size, width=img_size),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ])

    # 데이터 로드 및 분할
    df = pd.read_csv(os.path.join(data_path, "train_correct_labeling.csv"))
    train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

    train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform, cluster_dict=cluster_dict)
    val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform, cluster_dict=cluster_dict)
    test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

    # 각 클러스터에 대한 모델 학습
    for cluster in range(n_clusters):
        print(f"\nTraining model for cluster {cluster}")
        
        # 클러스터에 해당하는 데이터만 선택
        train_cluster = [data for data in train_dataset if data[2] == cluster]
        val_cluster = [data for data in val_dataset if data[2] == cluster]
        
        if len(train_cluster) == 0 or len(val_cluster) == 0:
            print(f"Skipping cluster {cluster} due to insufficient data")
            continue
        
        train_cluster_loader = DataLoader(train_cluster, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
        val_cluster_loader = DataLoader(val_cluster, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

        # Swin-B 모델 설정
        model = load_swin_b_model(num_classes=17).to(device)
        loss_fn = nn.CrossEntropyLoss()
        optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-5)
        scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

        # Early stopping 설정
        early_stopping = EarlyStopping(patience=5, min_delta=0.001, min_f1_score=0.95)


        # 모델 구조 출력
        # print(f"\nModel structure of Swin-B for cluster {cluster}:")
        # print_model_summary(model, (3, img_size, img_size))

        # 학습 루프
        best_val_f1 = 0
        for epoch in range(EPOCHS):
            train_loss, train_acc, train_f1 = train_one_epoch(train_cluster_loader, model, optimizer, loss_fn, device)
            val_loss, val_acc, val_f1 = validate(val_cluster_loader, model, loss_fn, device)
            scheduler.step()

            print(f"Epoch {epoch+1}/{EPOCHS}")
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

            if val_f1 > best_val_f1:
                best_val_f1 = val_f1
                torch.save(model.state_dict(), f"swin_b_model_cluster_{cluster}.pth")

            # Early stopping 체크
            early_stopping(val_loss, val_f1)
            if early_stopping.early_stop:
                print(f"Early stopping triggered at epoch {epoch+1} with F1 score: {val_f1:.4f}")
                break

    # 테스트 데이터 추론
    print("\nPerforming inference on test data")
    test_preds = []
    
    for cluster in range(n_clusters):
        model = load_swin_b_model(num_classes=17).to(device)
        model.load_state_dict(torch.load(f"swin_b_model_cluster_{cluster}.pth"))
        model.eval()
        
        cluster_preds = []
        for image, _, _ in tqdm(test_loader, desc=f"Predicting cluster {cluster}"):
            image = image.to(device)
            with torch.no_grad():
                preds = model(image)
            cluster_preds.extend(preds.detach().cpu().numpy())
        
        test_preds.append(cluster_preds)
    
    # 모든 클러스터의 예측을 결합
    final_preds = np.mean(test_preds, axis=0)
    final_preds = np.argmax(final_preds, axis=1)

    # 결과 저장
    pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
    pred_df['target'] = final_preds
    pred_df.to_csv("swin_pred.csv", index=False)
    print("Prediction completed and saved to pred.csv")


# 앙상블
CNN 모델(ConvNeXt V2 Large)과 Transformer 모델(ViT Large)을 결합한 앙상블 모델
- CNN 모델로 'convnextv2_large'를 사용합니다.
- Transformer 모델로 'vit_large_patch16_224'를 사용합니다.

## convnext v2 + vit

In [ ]:
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from torchsummary import summary
import gc

gc.collect()
torch.cuda.empty_cache()

# 데이터셋 클래스 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        if isinstance(csv, pd.DataFrame):
            self.df = csv.values
        else:
            self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = os.path.join(self.path, name)
        if not os.path.exists(img_path):
            print(f"Warning: Image not found: {img_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)
        else:
            img = np.array(Image.open(img_path).convert('RGB'))
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# 앙상블 모델 클래스 정의
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2

    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        return (out1 + out2) / 2

# 학습 함수 정의
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return train_loss, train_acc, train_f1

# 검증 함수 정의
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss = 0
    preds_list = []
    targets_list = []

    with torch.no_grad():
        for image, targets in loader:
            image = image.to(device)
            targets = targets.to(device)

            preds = model(image)
            loss = loss_fn(preds, targets)

            val_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average='macro')

    return val_loss, val_acc, val_f1

In [ ]:
# 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_path = '../data/'
img_size = 224
EPOCHS = 30
BATCH_SIZE = 16
LR = 1e-4
num_workers = 4

# 데이터 증강 설정
train_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, p=0.5),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# 데이터 로드 및 분할
df = pd.read_csv(os.path.join(data_path, "train.csv"))
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])

train_dataset = ImageDataset(train_df, os.path.join(data_path, "train_preprocessed/"), transform=train_transform)
val_dataset = ImageDataset(val_df, os.path.join(data_path, "train_preprocessed/"), transform=val_transform)
test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

# 모델 설정
model1 = timm.create_model('convnextv2_large', pretrained=True, num_classes=17)
model2 = timm.create_model('vit_large_patch16_224', pretrained=True, num_classes=17)

ensemble_model = EnsembleModel(model1, model2).to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(ensemble_model.parameters(), lr=LR, weight_decay=1e-5)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

# 학습 루프
best_val_f1 = 0
for epoch in range(EPOCHS):
    train_loss, train_acc, train_f1 = train_one_epoch(train_loader, ensemble_model, optimizer, loss_fn, device)
    val_loss, val_acc, val_f1 = validate(val_loader, ensemble_model, loss_fn, device)
    scheduler.step()

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(ensemble_model.state_dict(), "best_ensemble_model.pth")

# 테스트 데이터 추론
ensemble_model.load_state_dict(torch.load("best_ensemble_model.pth"))
ensemble_model.eval()
preds_list = []

for image, _ in tqdm(test_loader):
    image = image.to(device)
    with torch.no_grad():
        preds = ensemble_model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

# 결과 저장
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv("ensemble_pred.csv", index=False)
print("Prediction completed and saved to ensemble_pred.csv")

## 앙상블 모델 II - 리더 보드 제출용
CNN 모델(ConvNeXt V2 Large)과 Transformer 모델(Swin Transformers)을 결합한 앙상블 모델
- Hyper Parameter tunning이 전혀 되어 있지 않는 기본 모델 : 향후 최적화 필요
- CNN 모델로 'convnextv2_large'를 사용합니다.
- Transformer 모델로 'swin_large_patch4_window7_224'를 사용합니다.
- software voting(기존 저장된 pth 사용)

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import gc

gc.collect()
torch.cuda.empty_cache()

# 첫 번째 모델과 두 번째 모델 로드
model1 = timm.create_model('convnextv2_large', pretrained=False, num_classes=17).to(device)
model2 = timm.create_model('swin_large_patch4_window7_224', pretrained=False, num_classes=17).to(device)

# 모델 가중치 로드
model1.load_state_dict(torch.load('convNext_model.pth'))
model2.load_state_dict(torch.load('swin_t_model.pth'))

model1.eval()
model2.eval()

# 테스트 데이터 로드
test_dataset = ImageDataset(os.path.join(data_path, "sample_submission.csv"), os.path.join(data_path, "test_preprocessed/"), transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers, pin_memory=True)

# 소프트 보팅을 통한 예측
preds_list = []
with torch.no_grad():
    for image, _ in tqdm(test_loader):
        image = image.to(device)
        preds1 = model1(image)
        preds2 = model2(image)
        
        # 소프트 보팅: 예측 확률의 평균
        preds_avg = (torch.softmax(preds1, dim=1) + torch.softmax(preds2, dim=1)) / 2
        preds_list.extend(preds_avg.argmax(dim=1).detach().cpu().numpy())

# 결과 저장
pred_df = pd.DataFrame(test_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list
pred_df.to_csv("ensemble_pred.csv", index=False)
print("Ensemble prediction completed and saved to ensemble_pred.csv")


# 3가지 모델 앙상블